In [1]:
# !git clone https://github.com/RockatanskyV8/deep-learning.git
%cd deep-learning

/content/deep-learning


In [2]:
!git pull

Already up to date.


In [3]:
import pandas as pd
import numpy as np
import torch.optim as optim
from datetime import datetime
import random

from Plots  import *
from Modelos  import *

from Treinamento import Treinamento
import itertools

In [4]:
class Regressor():
    def __init__(self, dataset,  
                 batch_sizes    = [], 
                 learning_rates = [], 
                 weight_decays  = [], 
                 prob_desligar  = [],
                 neuronios      = [],
                 layers         = [],
                 epochs = 2000, early_stopping_epochs = 60):
      
        self.dataset               = dataset

        self.epochs                = epochs
        self.early_stopping_epochs = early_stopping_epochs

        self.batch_sizes           = batch_sizes
        self.learning_rates        = learning_rates
        self.weight_decays         = weight_decays
        self.prob_desligar         = prob_desligar
        self.neuronios             = neuronios
        self.layers                = layers

    def model_generator(self, neuronios, layers, prob, input_size = 3, output_size = 2):
        camadas = []
        for i in range(0, layers):
            if(int(neuronios/2**i) > 0):
              c = Camada( int(neuronios/2**i), nn.ReLU(), nn.Dropout(prob) )
              camadas.append(c)

        camadas.append(  Camada( output_size, nn.Softmax(dim=-1) ) )
        return GeradorRede(input_size, camadas)

    def regressao(self, criterion, best_model_path, best_global_model = None, input_size = 3):
        start = datetime.now()
        current_valid_loss = 0
        best_valid_loss = np.Inf
        total_accuracy = 0

        best_model = GeradorRede(input_size, [])

        for values in itertools.product(self.neuronios, self.learning_rates, self.batch_sizes, self.weight_decays, self.prob_desligar):

          tries_without_improv = 0

          for layers in self.layers:
            neuronios       = values[0]
            learning_rate   = values[1]
            batch_size      = values[2]
            wd              = values[3]
            p               = values[4]

            if tries_without_improv >= round(layers / 3 + 0.5):
                print(f'----------Tentativas sem melhora: {tries_without_improv}, Desligando...')
                break
            
            print(f'----------\nCamadas: {layers}')
            print(f'----------Learning rate: {learning_rate} ; Batch size: {batch_size}')
            print(f'Weight Decay: {wd} ; Probabilidades de desligamento:{p}\n----------\n')

            model = self.model_generator(neuronios, layers, p, input_size)

            if (best_global_model is not None):
              model.load_state_dict(torch.load(best_global_model))

            optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=wd)

            t = Treinamento(self.dataset, self.epochs, batch_size, self.early_stopping_epochs)
            model, train_loss, valid_loss = t.train_linear(model, optimizer, criterion)

            p = Plots()
            acc = p.get_accuracy(model, 
                                 torch.from_numpy(t.X_test), 
                                 torch.from_numpy(t.y_test.to_numpy()))

            lss = p.get_loss(model, 
                             torch.from_numpy(t.X_test), 
                             torch.from_numpy(t.y_test.to_numpy()), 
                             criterion)
            
            current_valid_loss = min(valid_loss)
            if current_valid_loss < best_valid_loss and total_accuracy < acc:
                torch.save(model.state_dict(), best_model_path)
                best_valid_loss = current_valid_loss
                total_accuracy = acc
                best_model = model
                print('--------------------------------------------------------- New best global model found! ---------------------------------------------------------')

                print(model)
                print(f'\nValidation loss: {current_valid_loss}\n')
                print(f'Accuracy: {acc}\n')
                print(f'Loss: {lss}\n')
            else:
              tries_without_improv += 1

            

          end = datetime.now()
          print(f'\n\n\n--------------------\nTotal training time: {end - start}')

        return best_valid_loss, total_accuracy, best_model


caso essa seja a chamada do método `model_generator( 500, 5, .5)`

esse será o modelo criado
```
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=500, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=500, out_features=250, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=250, out_features=125, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=125, out_features=62, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=62, out_features=31, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): Linear(in_features=31, out_features=2, bias=True)
    (16): Softmax(dim=-1)
  )
)
```

In [5]:
def run_models(dataset, input_features, best_model_path,
               num_neuronios = [], probs_desligar = [], weight_decays = [],
               num_layers = [], learning_rates = [], run_batch_sizes = [],
               run_epochs = 2000,
               run_early_stopping_epochs = 60,
               best_global_model = None):

  runner = Regressor(dataset,
                     learning_rates = learning_rates,
                     weight_decays  = weight_decays,
                     prob_desligar  = probs_desligar,
                     neuronios      = num_neuronios,
                     layers         = num_layers,
                     epochs = run_epochs,
                     batch_sizes = run_batch_sizes,
                     early_stopping_epochs = run_early_stopping_epochs )
  
  criterion = nn.CrossEntropyLoss()
  
  return runner.regressao(criterion, best_model_path, best_global_model) 


In [6]:
df = pd.read_csv('df_points.txt', sep='\t', index_col=[0])

# Tentativa 1

In [7]:
best_valid_loss, total_accuracy, best_model = run_models(df, 3, '../atp1',
            num_neuronios= [96, 112, 128, 144, 160], 
            probs_desligar= [.5, .25, .125, .062], 
            weight_decays= [1e-7, 1e-6, 1e-5, 1e-4],
            num_layers = [5, 6, 7, 8, 9],
            learning_rates = [0.9, 0.5], 
            run_batch_sizes = [100, 120],
            run_epochs = 2000,
            run_early_stopping_epochs = 75
            )

----------
Camadas: 5
----------Learning rate: 0.9 ; Batch size: 100
Weight Decay: 1e-07 ; Probabilidades de desligamento:0.5
----------



  7%|▋         | 137/2000 [00:36<08:21,  3.72it/s]


Training interrupted by early stopping!
Total epochs run: 138
Best model found at epoch 62 with valid loss 10.628925800323486 and training loss 56.37830060720444
Total training time: 0:00:36.871933
--------------------------------------------------------- New best global model found! ---------------------------------------------------------
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=96, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=96, out_features=48, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=48, out_features=24, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=24, out_features=12, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=12, out_features=6, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): Linear(in_features=6, out_featur

  4%|▍         | 82/2000 [00:21<08:13,  3.88it/s]


Training interrupted by early stopping!
Total epochs run: 83
Best model found at epoch 7 with valid loss 12.15892630815506 and training loss 54.698762118816376
Total training time: 0:00:21.124430
----------
Camadas: 7
----------Learning rate: 0.9 ; Batch size: 100
Weight Decay: 1e-07 ; Probabilidades de desligamento:0.5
----------



...

Training interrupted by early stopping!
Total epochs run: 141
Best model found at epoch 65 with valid loss 10.935009777545929 and training loss 51.89860612154007
Total training time: 0:00:41.013957
----------
Camadas: 7
----------Learning rate: 0.5 ; Batch size: 100
Weight Decay: 1e-05 ; Probabilidades de desligamento:0.5
----------



  4%|▍         | 77/2000 [00:24<10:10,  3.15it/s]


Training interrupted by early stopping!
Total epochs run: 78
Best model found at epoch 2 with valid loss 10.418465316295624 and training loss 49.59675979614258
Total training time: 0:00:24.448061
----------Tentativas sem melhora: 3, Desligando...



--------------------
Total training time: 1:05:03.633524
----------
Camadas: 5
----------Learning rate: 0.5 ; Batch size: 100
Weight Decay: 1e-05 ; Probabilidades de desligamento:0.25
----------



  3%|▎         | 54/2000 [00:14<08:45,  3.70it/s]


KeyboardInterrupt: ignored

# Tentativas picotadas

In [8]:
best_valid_loss1, total_accuracy1, best_model1 = run_models(df, 3, '../atp_pic1',
            num_neuronios= [96, 112], 
            probs_desligar= [.5, .25], 
            weight_decays= [1e-7, 1e-6],
            num_layers = [5, 6, 7, 8, 9],
            learning_rates = [0.09, 0.05], 
            run_batch_sizes = [100, 120],
            run_epochs = 2000,
            run_early_stopping_epochs = 75
            )

print(best_valid_loss1)
print(total_accuracy1)
print(best_model1)

----------
Camadas: 5
----------Learning rate: 0.09 ; Batch size: 100
Weight Decay: 1e-07 ; Probabilidades de desligamento:0.5
----------



  4%|▍         | 76/2000 [00:12<05:19,  6.03it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 12.238926231861115 and training loss 53.89424389600754
Total training time: 0:00:12.620230
--------------------------------------------------------- New best global model found! ---------------------------------------------------------
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=96, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=96, out_features=48, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=48, out_features=24, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=24, out_features=12, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=12, out_features=6, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): Linear(in_features=6, out_features

  5%|▍         | 91/2000 [00:19<06:40,  4.77it/s]


Training interrupted by early stopping!
Total epochs run: 92
Best model found at epoch 16 with valid loss 10.419057488441467 and training loss 49.20330232381821
Total training time: 0:00:19.105232
--------------------------------------------------------- New best global model found! ---------------------------------------------------------
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=96, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=96, out_features=48, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=48, out_features=24, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=24, out_features=12, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=12, out_features=6, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.5, inplace=False)
    (15): Linear(in_features=6, out_feature

...

Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 8.813751757144928 and training loss 40.72202146053314
Total training time: 0:00:19.541925
----------
Camadas: 6
----------Learning rate: 0.05 ; Batch size: 120
Weight Decay: 1e-06 ; Probabilidades de desligamento:0.25
----------



  4%|▍         | 76/2000 [00:23<09:42,  3.30it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 9.02262020111084 and training loss 41.07148718833923
Total training time: 0:00:23.043547
----------
Camadas: 7
----------Learning rate: 0.05 ; Batch size: 120
Weight Decay: 1e-06 ; Probabilidades de desligamento:0.25
----------



  4%|▍         | 77/2000 [00:20<08:43,  3.67it/s]


Training interrupted by early stopping!
Total epochs run: 78
Best model found at epoch 2 with valid loss 9.023422122001648 and training loss 40.93175905942917
Total training time: 0:00:20.984967
----------Tentativas sem melhora: 3, Desligando...



--------------------
Total training time: 0:37:31.663612
8.084309697151184
0.678
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=96, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=96, out_features=48, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.25, inplace=False)
    (6): Linear(in_features=48, out_features=24, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.25, inplace=False)
    (9): Linear(in_features=24, out_features=12, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.25, inplace=False)
    (12): Linear(in_features=12, out_features=6, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.25, inplace=False)
    (15): Linear(in_features=6, out_features=2, bi

In [9]:
best_valid_loss2, total_accuracy2, best_model2 = run_models(df, 3, '../atp_pic2',
            num_neuronios= [128, 144], 
            probs_desligar= [.125, .062], 
            weight_decays= [1e-5, 1e-4],
            num_layers = [5, 6, 7, 8, 9],
            learning_rates = [0.09, 0.05], 
            run_batch_sizes = [100, 120],
            run_epochs = 2000,
            run_early_stopping_epochs = 75
            )
print(best_valid_loss2)
print(total_accuracy2)
print(best_model2)

----------
Camadas: 5
----------Learning rate: 0.09 ; Batch size: 100
Weight Decay: 1e-05 ; Probabilidades de desligamento:0.125
----------



  6%|▌         | 119/2000 [00:48<12:41,  2.47it/s]


Training interrupted by early stopping!
Total epochs run: 120
Best model found at epoch 44 with valid loss 10.428783118724823 and training loss 54.09025377035141
Total training time: 0:00:48.210885
--------------------------------------------------------- New best global model found! ---------------------------------------------------------
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.125, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.125, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.125, inplace=False)
    (9): Linear(in_features=32, out_features=16, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.125, inplace=False)
    (12): Linear(in_features=16, out_features=8, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.125, inplace=False)
    (15): Linear(in_features=8

  4%|▍         | 76/2000 [00:31<13:19,  2.41it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 10.441726684570312 and training loss 48.70566713809967
Total training time: 0:00:31.618068
----------
Camadas: 7
----------Learning rate: 0.09 ; Batch size: 100
Weight Decay: 1e-05 ; Probabilidades de desligamento:0.125
----------



...

Training interrupted by early stopping!
Total epochs run: 146
Best model found at epoch 70 with valid loss 9.02142995595932 and training loss 40.97724986076355
Total training time: 0:01:15.120378
----------
Camadas: 6
----------Learning rate: 0.05 ; Batch size: 120
Weight Decay: 0.0001 ; Probabilidades de desligamento:0.062
----------



  6%|▌         | 118/2000 [01:05<17:23,  1.80it/s]


Training interrupted by early stopping!
Total epochs run: 119
Best model found at epoch 43 with valid loss 9.01958304643631 and training loss 40.94880920648575
Total training time: 0:01:05.447595
----------
Camadas: 7
----------Learning rate: 0.05 ; Batch size: 120
Weight Decay: 0.0001 ; Probabilidades de desligamento:0.062
----------



  8%|▊         | 150/2000 [00:56<11:39,  2.65it/s]


Training interrupted by early stopping!
Total epochs run: 151
Best model found at epoch 75 with valid loss 9.022847771644592 and training loss 40.94505190849304
Total training time: 0:00:56.721998
----------Tentativas sem melhora: 3, Desligando...



--------------------
Total training time: 1:10:28.122953
7.282630503177643
0.7133333333333334
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.125, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.125, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.125, inplace=False)
    (9): Linear(in_features=32, out_features=16, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.125, inplace=False)
    (12): Linear(in_features=16, out_features=8, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.125, inplace=False)
    (15): Linear(in_features

In [10]:
best_valid_loss3, total_accuracy3, best_model3 = run_models(df, 3, '../atp_pic3',
            num_neuronios= [160, 176], 
            probs_desligar= [.31, .16], 
            weight_decays= [1e-7, 1e-6],
            num_layers = [5, 6, 7, 8, 9],
            learning_rates = [0.09, 0.05], 
            run_batch_sizes = [100, 120],
            run_epochs = 2000,
            run_early_stopping_epochs = 75
            )

print(best_valid_loss3)
print(total_accuracy3)
print(best_model3)

----------
Camadas: 5
----------Learning rate: 0.09 ; Batch size: 100
Weight Decay: 1e-07 ; Probabilidades de desligamento:0.31
----------



  7%|▋         | 136/2000 [00:38<08:54,  3.49it/s]


Training interrupted by early stopping!
Total epochs run: 137
Best model found at epoch 61 with valid loss 10.405647814273834 and training loss 50.68290710449219
Total training time: 0:00:39.003910
--------------------------------------------------------- New best global model found! ---------------------------------------------------------
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=160, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.31, inplace=False)
    (3): Linear(in_features=160, out_features=80, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.31, inplace=False)
    (6): Linear(in_features=80, out_features=40, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.31, inplace=False)
    (9): Linear(in_features=40, out_features=20, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.31, inplace=False)
    (12): Linear(in_features=20, out_features=10, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.31, inplace=False)
    (15): Linear(in_features=10, o

  9%|▉         | 181/2000 [01:09<11:37,  2.61it/s]


Training interrupted by early stopping!
Total epochs run: 182
Best model found at epoch 106 with valid loss 10.40170305967331 and training loss 49.573061883449554
Total training time: 0:01:09.388185
----------
Camadas: 7
----------Learning rate: 0.09 ; Batch size: 100
Weight Decay: 1e-07 ; Probabilidades de desligamento:0.31
----------



...

Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 7.862796723842621 and training loss 38.9080713391304
Total training time: 0:00:26.604740
----------
Camadas: 6
----------Learning rate: 0.05 ; Batch size: 120
Weight Decay: 1e-06 ; Probabilidades de desligamento:0.16
----------



  4%|▍         | 76/2000 [00:40<17:14,  1.86it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 9.022698640823364 and training loss 40.98201656341553
Total training time: 0:00:40.883937
----------
Camadas: 7
----------Learning rate: 0.05 ; Batch size: 120
Weight Decay: 1e-06 ; Probabilidades de desligamento:0.16
----------



  4%|▍         | 76/2000 [00:40<17:06,  1.87it/s]


Training interrupted by early stopping!
Total epochs run: 77
Best model found at epoch 1 with valid loss 8.86769163608551 and training loss 40.66466635465622
Total training time: 0:00:40.549253
----------Tentativas sem melhora: 3, Desligando...



--------------------
Total training time: 0:55:29.008804
8.78389972448349
0.694
GeradorRede(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=160, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.16, inplace=False)
    (3): Linear(in_features=160, out_features=80, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.16, inplace=False)
    (6): Linear(in_features=80, out_features=40, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.16, inplace=False)
    (9): Linear(in_features=40, out_features=20, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.16, inplace=False)
    (12): Linear(in_features=20, out_features=10, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.16, inplace=False)
    (15): Linear(in_features=10, out_features=2, 